# Data Wraggling/Cleaning
---

In [1]:
# Setup dependencies
import pandas as pd
from pathlib import Path

# Set up file paths
file_covid_counties = Path('../Resources/covid_counties_2022.csv')
file_educational_data = Path('../Resources/Project_1_Educational_Data.csv')
file_state_names = Path('../Resources/state_names.csv')
file_va_poverty = Path('../Resources/PovertyReport_2021.csv')

# Read into the data
# Note: the state_names will be used to merge with census/edu attainment dataset
covid_counties = pd.read_csv(file_covid_counties)
educational_data = pd.read_csv(file_educational_data)
state_names = pd.read_csv(file_state_names)
va_counties_poverty = pd.read_csv(file_va_poverty)

# Print the column headers to look for columns of interest
print(covid_counties.columns)
print()
print(educational_data.columns)
print()
print(state_names.columns)
print()
print(va_counties_poverty.columns)

Index(['fips', 'country', 'state', 'county', 'level', 'lat', 'locationId',
       'long', 'population', 'metrics.testPositivityRatio',
       'metrics.testPositivityRatioDetails.source', 'metrics.caseDensity',
       'metrics.contactTracerCapacityRatio', 'metrics.infectionRate',
       'metrics.infectionRateCI90', 'unused1', 'unused2',
       'metrics.icuCapacityRatio', 'riskLevels.overall',
       'riskLevels.testPositivityRatio', 'riskLevels.caseDensity',
       'riskLevels.contactTracerCapacityRatio', 'riskLevels.infectionRate',
       'unused3', 'riskLevels.icuCapacityRatio', 'actuals.cases',
       'actuals.deaths', 'actuals.positiveTests', 'actuals.negativeTests',
       'actuals.contactTracers', 'actuals.hospitalBeds.capacity',
       'actuals.hospitalBeds.currentUsageTotal',
       'actuals.hospitalBeds.currentUsageCovid', 'unused4',
       'actuals.icuBeds.capacity', 'actuals.icuBeds.currentUsageTotal',
       'actuals.icuBeds.currentUsageCovid', 'unused5', 'actuals.newCases',

In [2]:
# Create new DF for covid counties dataset
df_covid_counties_cleaned = pd.DataFrame(covid_counties[[
    'fips', 'state', 'county', 'population', 'actuals.cases', 'actuals.deaths', 'actuals.hospitalBeds.capacity',
    'actuals.hospitalBeds.currentUsageTotal', 'actuals.hospitalBeds.currentUsageCovid', 'actuals.icuBeds.capacity',
    'actuals.icuBeds.currentUsageTotal', 'actuals.icuBeds.currentUsageCovid', 'actuals.vaccinationsInitiated',
    'actuals.vaccinationsCompleted', 'metrics.vaccinationsInitiatedRatio', 'metrics.vaccinationsCompletedRatio',
    'actuals.vaccinationsAdditionalDose', 'metrics.vaccinationsAdditionalDoseRatio',
    'actuals.vaccinationsFall2022BivalentBooster', 'metrics.vaccinationsFall2022BivalentBoosterRatio'
]])

# Rename the columns
df_covid_counties_cleaned = df_covid_counties_cleaned.rename(columns = {
    'actuals.cases': 'total cases',
    'actuals.deaths': 'total deaths',
    'actuals.hospitalBeds.capacity': 'hospital beds',
    'actuals.hospitalBeds.currentUsageTotal': 'hospital beds usage',
    'actuals.hospitalBeds.currentUsageCovid': 'hospital beds covid',
    'actuals.icuBeds.capacity': 'hospital icu beds',
    'actuals.icuBeds.currentUsageTotal': 'hospital icu beds usage',
    'actuals.icuBeds.currentUsageCovid': 'hospital icu beds covid',
    'actuals.vaccinationsInitiated': 'vaccination initiated',
    'actuals.vaccinationsCompleted': 'vaccination completed',
    'metrics.vaccinationsInitiatedRatio': 'vaccination initiated ratio',
    'metrics.vaccinationsCompletedRatio': 'vaccination completed ratio',
    'actuals.vaccinationsAdditionalDose': 'vaccination addl dose',
    'metrics.vaccinationsAdditionalDoseRatio': 'vaccination addl dose ratio',
    'actuals.vaccinationsFall2022BivalentBooster': 'vaccination booster',
    'metrics.vaccinationsFall2022BivalentBoosterRatio': 'vaccination booster ratio'
})

# Replace NaN values with 0 by looping through each column and fill in 0 for NaN
for col in df_covid_counties_cleaned:
    if (col != 'fips') & (col != 'state') & (col != 'county'):
        df_covid_counties_cleaned[col] = df_covid_counties_cleaned[col].fillna(0)

# Display the DF
df_covid_counties_cleaned.head()

,fips,state,county,population,total cases,total deaths,hospital beds,hospital beds usage,hospital beds covid,hospital icu beds,hospital icu beds usage,hospital icu beds covid,vaccination initiated,vaccination completed,vaccination initiated ratio,vaccination completed ratio,vaccination addl dose,vaccination addl dose ratio,vaccination booster,vaccination booster ratio
0,2013,AK,Aleutians East Borough,3337,812,0,0.0,0.0,0.0,0.0,0.0,0.0,2900.0,2498.0,0.869,0.749,1038.0,0.311,199.0,0.060
1,2016,AK,Aleutians West Census Area,5634,1174,0,0.0,0.0,0.0,0.0,0.0,0.0,4410.0,3637.0,0.783,0.646,1425.0,0.253,500.0,0.089
2,2020,AK,Anchorage Municipality,288000,121792,284,559.0,486.0,12.0,60.0,42.0,1.0,230587.0,205043.0,0.801,0.712,103924.0,0.361,47001.0,0.163
3,2050,AK,Bethel Census Area,18386,13328,0,0.0,0.0,0.0,0.0,0.0,0.0,13690.0,12693.0,0.745,0.690,6688.0,0.364,1806.0,0.098
4,2060,AK,Bristol Bay Borough,836,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.000


In [3]:
# Data cleaning for education attainment dataset
# Split the combined county and state column into county and state columns
educational_data[['County', 'State']] = educational_data['County'].str.split(', ', expand = True)
educational_data = educational_data[['County', 'State', 'Population 18-24', 'Bachelors 18-24', 
                                           'Population Over 25', 'Bachelors Over 25']]

# Display the DF
educational_data.head()

,County,State,Population 18-24,Bachelors 18-24,Population Over 25,Bachelors Over 25
0,Baldwin County,Alabama,16407,1620,178759,59850
1,Calhoun County,Alabama,12900,985,78478,16568
2,Cullman County,Alabama,7022,112,63473,12787
3,DeKalb County,Alabama,6580,429,48375,6597
4,Elmore County,Alabama,6731,123,63862,14345


In [4]:
# Merge the df_educational_data with the state_names by the State
df_educational_state_names_merged = pd.merge(educational_data, state_names[['State', 'Alpha code']], 
                                             how = 'left', on = 'State')

df_educational_state_names_merged = df_educational_state_names_merged[[
    'County', 'State', 'Alpha code', 'Population 18-24', 'Bachelors 18-24', 'Population Over 25', 'Bachelors Over 25'
]]

# Display the educational DF merged with the state_names
df_educational_state_names_merged.head()


,County,State,Alpha code,Population 18-24,Bachelors 18-24,Population Over 25,Bachelors Over 25
0,Baldwin County,Alabama,AL,16407,1620,178759,59850
1,Calhoun County,Alabama,AL,12900,985,78478,16568
2,Cullman County,Alabama,AL,7022,112,63473,12787
3,DeKalb County,Alabama,AL,6580,429,48375,6597
4,Elmore County,Alabama,AL,6731,123,63862,14345


In [5]:
# Create a new DF by merging the census education attainment and covid by counties datasets by the county and state
# using the left join method for the census dataset.
#
# Note: extract only wanted columns specific to the analysis between census and covid datasets 
df_covid_educational_merged = pd.merge(df_educational_state_names_merged, df_covid_counties_cleaned,
                                      how = 'inner', 
                                      left_on = ['County', 'Alpha code'], 
                                      right_on = ['county', 'state'])

# Move the fips column to the first column position
col_to_move = df_covid_educational_merged.pop('fips')
df_covid_educational_merged.insert(0, 'fips', col_to_move)

# Drop the extra columns
df_covid_educational_merged = df_covid_educational_merged.drop(columns = ['state', 'county'])

# Display the merged DF
df_covid_educational_merged.head()

,fips,County,State,Alpha code,Population 18-24,Bachelors 18-24,Population Over 25,Bachelors Over 25,population,total cases,...,hospital icu beds usage,hospital icu beds covid,vaccination initiated,vaccination completed,vaccination initiated ratio,vaccination completed ratio,vaccination addl dose,vaccination addl dose ratio,vaccination booster,vaccination booster ratio
0,1003,Baldwin County,Alabama,AL,16407,1620,178759,59850,223234,70476,...,26.0,0.0,149596.0,119284.0,0.670,0.534,47583.0,0.213,19133.0,0.086
1,1015,Calhoun County,Alabama,AL,12900,985,78478,16568,113605,41912,...,18.0,1.0,67165.0,55615.0,0.591,0.490,22007.0,0.194,7679.0,0.068
2,1043,Cullman County,Alabama,AL,7022,112,63473,12787,83768,31702,...,7.0,1.0,38911.0,32943.0,0.465,0.393,13129.0,0.157,4396.0,0.052
3,1049,DeKalb County,Alabama,AL,6580,429,48375,6597,71513,22651,...,3.0,1.0,30901.0,25210.0,0.432,0.353,9896.0,0.138,3166.0,0.044
4,1051,Elmore County,Alabama,AL,6731,123,63862,14345,81209,30060,...,2.0,0.0,48321.0,39299.0,0.595,0.484,14703.0,0.181,5098.0,0.063


In [6]:
# Create a cleaned DF for the va_counties_poverty and just use the columns we want to see
df_va_poverty_cleaned = va_counties_poverty[['fips2', 'county2', 'total_est_pct3']].rename(columns = {
    'fips2': 'fips',
    'county2': 'county',
    'total_est_pct3': 'total poverty percentage'
})

# Print the DF
df_va_poverty_cleaned

,fips,county,total poverty percentage
0,51107,Loudoun,3.6
1,51610,Falls Church,4.0
2,51127,New Kent,4.9
3,51199,York,5.1
4,51179,Stafford,5.4
...,...,...,...
129,51590,Danville,24.2
130,51081,Greensville,24.4
131,51660,Harrisonburg,25.0
132,51105,Lee,25.1


In [7]:
# Merge the DF between covid dataset and VA 2021 dataset using inner join on fips
df_covid_poverty_merged = pd.merge(df_va_poverty_cleaned, df_covid_counties_cleaned,
                                  how = 'inner', on = 'fips')

# Cleanup and reorganize the merged DF
df_covid_poverty_merged_cleaned = df_covid_poverty_merged[[
    'fips', 'county_x', 'total poverty percentage', 'population', 'total cases',
    'vaccination completed', 'vaccination completed ratio'
]]
df_covid_poverty_merged_cleaned = df_covid_poverty_merged_cleaned.rename(columns = {'county_x': 'county'})

# Display the cleaned DF
df_covid_poverty_merged_cleaned

,fips,county,total poverty percentage,population,total cases,vaccination completed,vaccination completed ratio
0,51107,Loudoun,3.6,413538,98652,330884.0,0.800
1,51610,Falls Church,4.0,14617,3543,11828.0,0.809
2,51127,New Kent,4.9,23091,6015,15063.0,0.652
3,51199,York,5.1,68280,14008,58642.0,0.859
4,51179,Stafford,5.4,152882,42534,102448.0,0.670
...,...,...,...,...,...,...,...
128,51590,Danville,24.2,40044,14967,22273.0,0.556
129,51081,Greensville,24.4,11336,3928,6141.0,0.542
130,51660,Harrisonburg,25.0,53016,15367,29908.0,0.564
131,51105,Lee,25.1,23423,8538,11987.0,0.512


In [8]:
# Export the DFs into csv files for use in analysis files and store them in the Outputs folder
df_covid_counties_cleaned.to_csv('../Outputs/cleaned_covid_counties.csv', index = False)
df_covid_educational_merged.to_csv('../Outputs/cleaned_educational_covid_merged.csv', index = False)
df_covid_poverty_merged_cleaned.to_csv('../Outputs/cleaned_va_poverty_covid_merged.csv', index = False)